In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys, getopt
import csv
import pandas as pd
from collections import Counter
import re
import time
import timeit
import numpy as np
from scipy import stats
import scipy.stats.distributions as dist
import researchpy as rp

In [2]:
import os
path = os.getcwd()
print(path)
os.chdir(r"C:\Users\stanma02\Desktop\Final Project\WebScraper and Data")
path = os.getcwd()
print(path)

C:\Users\stanma02\Desktop\Final Project\WebScraper and Data
C:\Users\stanma02\Desktop\Final Project\WebScraper and Data


In [59]:
df = pd.read_csv("Match_Report_URLs_Total.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("-",expand=True)
df.head()

,Match_ID,League,Season,Wk,Day,Date,Time,Home,Score,Away,...,Venue,Referee,Match Report,Notes,Venue Key,VAR Key,Stadium_Capacity,VAR,HomeGoal,AwayGoal
0,1,Bundesliga,16-17,1,Fri,8/26/2016,20:30 (14:30),Bayern Munich,6-0,Werder Bremen,...,Allianz Arena,Christian Dingert,https://fbref.com/en/matches/cd4ef1df/Bayern-M...,NaN,Bayern Munich Allianz Arena,Bundesliga 16-17,75000,0,6,0
1,2,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Dortmund,2-1,Mainz 05,...,Signal Iduna Park,Robert Hartmann,https://fbref.com/en/matches/0a2c80cb/Dortmund...,NaN,Dortmund Signal Iduna Park,Bundesliga 16-17,81365,0,2,1
2,3,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Hamburger SV,1-1,Ingolstadt 04,...,Volksparkstadion,Marco Fritz,https://fbref.com/en/matches/58ae4021/Hamburge...,NaN,Hamburger SV Volksparkstadion,Bundesliga 16-17,57000,0,1,1
3,4,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Eint Frankfurt,1-0,Schalke 04,...,Commerzbank-Arena,Wolfgang Stark,https://fbref.com/en/matches/75808fc7/Eintrach...,NaN,Eint Frankfurt Commerzbank-Arena,Bundesliga 16-17,51500,0,1,0
4,5,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Koln,2-0,Darmstadt 98,...,RheinEnergieSTADION,Patrick Ittrich,https://fbref.com/en/matches/a2d4e20f/Koln-Dar...,NaN,Koln RheinEnergieSTADION,Bundesliga 16-17,49698,0,2,0


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9130 entries, 0 to 9129
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Match_ID          9130 non-null   int64  
 1   League            9130 non-null   object 
 2   Season            9130 non-null   object 
 3   Wk                9130 non-null   int64  
 4   Day               9130 non-null   object 
 5   Date              9130 non-null   object 
 6   Time              9057 non-null   object 
 7   Home              9130 non-null   object 
 8   Score             9029 non-null   object 
 9   Away              9130 non-null   object 
 10  Attendance        6897 non-null   float64
 11  Venue             9130 non-null   object 
 12  Referee           9029 non-null   object 
 13  Match Report      9130 non-null   object 
 14  Notes             2 non-null      object 
 15  Venue Key         9130 non-null   object 
 16  VAR Key           9130 non-null   object 


In [61]:
def homeresult(c):
    if c['HomeGoal']>c['AwayGoal']:
        return 'W'
    elif c['HomeGoal']<c['AwayGoal']:
        return 'L'
    else:
        return 'T'
df["HomeResult"]= df.apply(homeresult,axis=1)

In [62]:
df['Attendance'].fillna(0, inplace=True)
df['StadiumUtilization']=df['Attendance']/df['Stadium_Capacity']

In [63]:
def Utilization(c):
    if c['StadiumUtilization']> 1:
        return 1
    else:
        return c['StadiumUtilization']
df['StadiumUtilization']= df.apply(Utilization,axis=1)

In [64]:
def UtilizationBuckets(c):
    if c['StadiumUtilization']== 1:
        return 'Full'
    elif c['StadiumUtilization'] < 1 and c['StadiumUtilization'] >= .75:
        return '75%-99%'
    elif c['StadiumUtilization'] < .75 and c['StadiumUtilization'] >= .50:
        return '50%-74%'
    elif c['StadiumUtilization'] < .5 and c['StadiumUtilization'] >= .25:
        return '25%-49%'
    elif c['StadiumUtilization'] < .25 and c['StadiumUtilization'] >= 0.000000000000000001:
        return '.01%-24%'
    else:
        return 'No Fans'
df['UtilizationBuckets']= df.apply(UtilizationBuckets,axis=1)   

In [65]:
Games= df[df['Match Report']!= "Match Cancelled"]

In [66]:
Games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9029 entries, 0 to 9129
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Match_ID            9029 non-null   int64  
 1   League              9029 non-null   object 
 2   Season              9029 non-null   object 
 3   Wk                  9029 non-null   int64  
 4   Day                 9029 non-null   object 
 5   Date                9029 non-null   object 
 6   Time                9029 non-null   object 
 7   Home                9029 non-null   object 
 8   Score               9029 non-null   object 
 9   Away                9029 non-null   object 
 10  Attendance          9029 non-null   float64
 11  Venue               9029 non-null   object 
 12  Referee             9029 non-null   object 
 13  Match Report        9029 non-null   object 
 14  Notes               2 non-null      object 
 15  Venue Key           9029 non-null   object 
 16  VAR Ke

In [67]:
df= Games[Games['Notes'].isna()]

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9027 entries, 0 to 9129
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Match_ID            9027 non-null   int64  
 1   League              9027 non-null   object 
 2   Season              9027 non-null   object 
 3   Wk                  9027 non-null   int64  
 4   Day                 9027 non-null   object 
 5   Date                9027 non-null   object 
 6   Time                9027 non-null   object 
 7   Home                9027 non-null   object 
 8   Score               9027 non-null   object 
 9   Away                9027 non-null   object 
 10  Attendance          9027 non-null   float64
 11  Venue               9027 non-null   object 
 12  Referee             9027 non-null   object 
 13  Match Report        9027 non-null   object 
 14  Notes               0 non-null      object 
 15  Venue Key           9027 non-null   object 
 16  VAR Ke

In [69]:
df['Season'].value_counts()

18-19    1826
17-18    1826
20-21    1825
16-17    1825
19-20    1725
Name: Season, dtype: int64

In [70]:
df['League'].value_counts()

EPL           1900
LaLiga        1900
SerieA        1898
Ligue1        1799
Bundesliga    1530
Name: League, dtype: int64

In [76]:
Table= pd.crosstab(df.League,df.Season)
Table

Season,16-17,17-18,18-19,19-20,20-21
League,,,,,
Bundesliga,306,306,306,306,306
EPL,380,380,380,380,380
LaLiga,380,380,380,380,380
Ligue1,380,380,380,279,380
SerieA,379,380,380,380,379


In [71]:
Table= pd.crosstab(df.League,df.VAR)
Table

VAR,0,1
League,,
Bundesliga,306,1224
EPL,760,1140
LaLiga,760,1140
Ligue1,760,1039
SerieA,379,1519


In [72]:
df['UtilizationBuckets'].value_counts()

75%-99%     3484
No Fans     2131
50%-74%     1970
25%-49%      699
Full         569
.01%-24%     174
Name: UtilizationBuckets, dtype: int64

In [73]:
Table= pd.crosstab(df.League,df.UtilizationBuckets)
Table

UtilizationBuckets,.01%-24%,25%-49%,50%-74%,75%-99%,Full,No Fans
League,,,,,,
Bundesliga,36,11,108,647,375,353
EPL,31,7,21,1265,136,440
LaLiga,9,110,684,601,8,488
Ligue1,14,271,556,564,10,384
SerieA,84,300,601,407,40,466


In [74]:
Table= pd.crosstab(df.League,df.HomeResult)
Table

HomeResult,L,T,W
League,,,
Bundesliga,472,379,679
EPL,614,429,857
LaLiga,541,499,860
Ligue1,534,466,799
SerieA,619,452,827


In [78]:
Table= pd.crosstab(df.UtilizationBuckets,df.HomeResult)
Table

HomeResult,L,T,W
UtilizationBuckets,,,
.01%-24%,57,38,79
25%-49%,182,165,352
50%-74%,552,514,904
75%-99%,1077,840,1567
Full,170,129,270
No Fans,742,539,850


In [79]:
Table= pd.crosstab(df.VAR,df.HomeResult)
Table

HomeResult,L,T,W
VAR,,,
0,849,703,1413
1,1931,1522,2609


In [75]:
df.head()

,Match_ID,League,Season,Wk,Day,Date,Time,Home,Score,Away,...,Notes,Venue Key,VAR Key,Stadium_Capacity,VAR,HomeGoal,AwayGoal,HomeResult,StadiumUtilization,UtilizationBuckets
0,1,Bundesliga,16-17,1,Fri,8/26/2016,20:30 (14:30),Bayern Munich,6-0,Werder Bremen,...,NaN,Bayern Munich Allianz Arena,Bundesliga 16-17,75000,0,6,0,W,1.000000,Full
1,2,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Dortmund,2-1,Mainz 05,...,NaN,Dortmund Signal Iduna Park,Bundesliga 16-17,81365,0,2,1,W,0.999939,75%-99%
2,3,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Hamburger SV,1-1,Ingolstadt 04,...,NaN,Hamburger SV Volksparkstadion,Bundesliga 16-17,57000,0,1,1,T,0.879070,75%-99%
3,4,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Eint Frankfurt,1-0,Schalke 04,...,NaN,Eint Frankfurt Commerzbank-Arena,Bundesliga 16-17,51500,0,1,0,W,0.912621,75%-99%
4,5,Bundesliga,16-17,1,Sat,8/27/2016,15:30 (09:30),Koln,2-0,Darmstadt 98,...,NaN,Koln RheinEnergieSTADION,Bundesliga 16-17,49698,0,2,0,W,1.000000,Full
